In [1]:
## Below code required to access Google drive resources
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Keras and TF imports
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

In [0]:
# Scikit-learn and Numpy imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

In [0]:
# set the location of the training and test images (change as required)
location_of_train_data = "drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train"
location_of_test_data = "drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_test"

In [0]:
class_names = ["bishop", "king", "knight", "pawn", "queen", "rook", "empty"]
class_names_reverse_mappings = {"bishop": 0, "king": 1, "knight":2, "pawn":3, "queen":4, "rook":5, "empty":6}
class_names_folder_mappings = {"bishop": ["bb", "wb"], "king": ["bk", "wk"], "knight":["bn", "wn"], "pawn":["bp", "wp"], "queen":["bq", "wq"], "rook":["br", "wr"], "empty":["empty"]}

In [0]:
from keras.applications.inception_v3 import preprocess_input

def get_required_data_with_labels_for_model(base_location, dimensions=(299, 299)):
  X, y = [], []
  for class_name in class_names_folder_mappings:
    for folder_name in class_names_folder_mappings[class_name]:
      complete_path = os.path.join(base_location, folder_name)
      print("Reading the files from the location {0}".format(complete_path))
      
      for image_file_name in os.listdir(complete_path):
        
        # check if the current file is an image file with jpg extension
        if image_file_name.endswith(".jpg"):
          img_path = os.path.join(complete_path, image_file_name)
          
          # basic pre-processing of the images
          img = image.load_img(img_path, target_size=dimensions)
          x = image.img_to_array(img)
          x = preprocess_input(x)
          
          X.append(x)
          class_name_id = class_names_reverse_mappings[class_name]
          y.append(class_name_id)
  
  return X, y    
  

In [0]:
X_train, y_train = get_required_data_with_labels_for_model(location_of_train_data)
print(len(X_train))
print(len(y_train))
print(X_train[0].shape)
print(y_train[0])

Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/bb
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/wb
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/bk
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/wk
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/bn
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/wn
Reading the files from the location drive/My Drive/Extending board games online/chess_pieces_data/Chess ID Public Data/output_train/bp
Reading the files from the location drive/My Drive/Exte

In [0]:
X_test, y_test = get_required_data_with_labels_for_model(location_of_test_data)
print(len(X_test))
print(len(y_test))
print(X_test[0].shape)
print(y_test[0])

In [0]:
## update the base inception v3 model

num_output_classes = len(class_names)

# create the base pre-trained model
inception_v3_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = inception_v3_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_output_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=inception_v3_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in inception_v3_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [0]:
# checkpoint
filepath="drive/My Drive/Extending board games online/models/chess_pieces_inceptionv3_p1_0912.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_acc', patience=25, min_delta= 0.01)
callbacks_list = [checkpoint, early_stopping]

In [0]:
epochs = 500
batch_size = 500

In [0]:
history = model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1,
          callbacks=callbacks_list,
          batch_size=batch_size)

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
## Fine tune some inception layers
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [0]:
# checkpoint
filepath="drive/My Drive/Extending board games online/models/chess_pieces_inceptionv3_p2_0912.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_acc', patience=25, min_delta= 0.01)
callbacks_list = [checkpoint, early_stopping]

In [0]:
epochs = 200
batch_size = 500

In [0]:
history_2 = model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1,
          callbacks=callbacks_list,
          batch_size=batch_size)

In [0]:
# summarize history for accuracy
plt.plot(history_2.history['acc'])
plt.plot(history_2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
## Helper method to print a confusion matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [0]:
model.load_weights("drive/My Drive/Extending board games online/models/chess_pieces_inceptionv3_p2_0912.hdf5")

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=SGD(lr=0.0001, momentum=0.9),
    metrics=['accuracy'],
)
score = model.evaluate(X_test, y_test, verbose=0)

print("Score: " + str(score))

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

test_predictions = model.predict(X_test, batch_size=batch_size)
y_test_pred = [np.argmax(x) for x in test_predictions]
cnf_matrix = confusion_matrix(y_test, y_test_pred)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,title='Normalized confusion matrix')